In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(os.path.join(os.getcwd(), ".."))
sys.path

['/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python310.zip',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/lib-dynload',
 '',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/site-packages',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/../../..',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/../..',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/..']

In [2]:
import pandas as pd
import torchvision.datasets
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
from torch.nn import Conv2d
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
import tqdm
from utils import utils, nn_utils

In [30]:
class CNN_2D_Model(nn.Module):
    def __init__(self, n_classes, N, n_filters, kernel_size, stride, img_size):
        super(CNN_2D_Model, self).__init__()
        # padding: same ensures the output has the same size as the input
        self.conv2d = Conv2d(in_channels=3,
                             out_channels=n_filters,
                             kernel_size=kernel_size,
                             stride=stride,
                             padding="same")
        self.conv2d_hidden = Conv2d(in_channels=n_filters,
                                    out_channels=n_filters,
                                    kernel_size=kernel_size,
                                    stride=stride,
                                    padding="same")
        # intermediate hidden layers (number = N-1): hidden_dim --> hidden_dim
        # N-1 because we already have one layer converting input_dim --> hidden_dim
        self.conv2d_hidden_layers = nn_utils.create_clones(self.conv2d_hidden, N - 1)
        self.linear = nn.Linear(img_size * img_size * n_filters, n_classes)

    def forward(self, X):
        X = F.relu(self.conv2d(X))
        for conv2d_hidden_layer in self.conv2d_hidden_layers:
            X = F.relu(conv2d_hidden_layer(X))

        # aggregate the embeddings from cnn
        # mean of the representations of all tokens
        self.cnn_emb = torch.flatten(X, 1)  # flatten all dimensions except batch
        y = self.linear(self.cnn_emb)
        return y


def get_cnn_model(model):
    cnn_model = CNN_2D_Model(n_classes=model["n_classes"],
                             N=model["depth"],
                             n_filters=model["n_filters"],
                             kernel_size=model["kernel_size"],
                             stride=model["stride"],
                             img_size=model["img_size"])

    print(cnn_model)
    print("Number of parameters = ", sum(p.numel() for p in cnn_model.parameters() if p.requires_grad))
    return cnn_model.to(nn_utils.get_device())

In [33]:
def run_model(model, train_dataset_loader, test_dataset_loader, loss, n_epochs, model_name, mode):
    tbw = SummaryWriter()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    lr_scheduler = OneCycleLR(
        optimizer=optimizer,
        max_lr=1e-4,
        epochs=n_epochs,
        steps_per_epoch=len(train_dataset_loader),
        pct_start=0.1,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=10000.0)
    model.train_iter = 0
    model.test_iter = 0
    if mode == "train":
        # train the model only if set to train mode
        for e in range(n_epochs):
            model = run_epoch(model, train_dataset_loader, test_dataset_loader, criterion, optimizer,
                              lr_scheduler, tbw, model_name, e)

    return evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch=None, log_loss=False), model


def run_epoch(model, train_dataset_loader, test_dataset_loader, criterion, optimizer, lr_scheduler, tbw, model_name,
              epoch):
    # Training
    model.train()
    for _, record in enumerate(pbar := tqdm.tqdm(train_dataset_loader)):
        input, label = record
        input = input.to(nn_utils.get_device())
        label = label.to(nn_utils.get_device())
        optimizer.zero_grad()

        output = model(input)
        output = output.to(nn_utils.get_device())

        loss = criterion(output, label.long())
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        model.train_iter += 1
        curr_lr = lr_scheduler.get_last_lr()[0]
        train_loss = loss.item()
        tbw.add_scalar(f"{model_name}/learning-rate", float(curr_lr), model.train_iter)
        tbw.add_scalar(f"{model_name}/training-loss", float(train_loss), model.train_iter)
        pbar.set_description(
            f"{model_name}/training-loss = {float(train_loss)}, model.n_iter={model.train_iter}, epoch={epoch + 1}")

    # Testing
    evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch, log_loss=True)
    return model


def evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch, log_loss=False):
    with torch.no_grad():
        model.eval()

        results = []
        for _, record in enumerate(pbar := tqdm.tqdm(test_dataset_loader)):
            input, label = record
            input = input.to(nn_utils.get_device())
            label = label.to(nn_utils.get_device())
            output = model(input)  # b x n_classes
            output = output.to(nn_utils.get_device())

            loss = criterion(output, label.long())
            val_loss = loss.item()
            model.test_iter += 1
            if log_loss:
                tbw.add_scalar(f"{model_name}/validation-loss", float(val_loss), model.test_iter)
                pbar.set_description(
                    f"{model_name}/validation-loss = {float(val_loss)}, model.n_iter={model.test_iter}, epoch={epoch + 1}")
            # to get probabilities of the output
            output = F.softmax(output, dim=-1)
            result_df = pd.DataFrame(output.cpu().numpy())
            result_df["y_true"] = label.cpu().numpy()
            results.append(result_df)
    return pd.concat(results, ignore_index=True)


def main():
    print("Main")
    transform = transforms.Compose([transforms.ToTensor()])
    batch_size = 64
    train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    model = get_cnn_model({
        "n_classes": 10,
        "depth": 2,
        "n_filters": 3,
        "kernel_size": 3,
        "stride": 1,
        "img_size": 32
    })

    results_df, _ = run_model(model=model,
              train_dataset_loader=train_dataset_loader,
              test_dataset_loader=test_dataset_loader,
              loss="FocalLoss",
              n_epochs=10,
              model_name="CNN-CIFAR10",
              mode="train")
    return results_df

In [34]:
results_df = main()

Main
Files already downloaded and verified
Files already downloaded and verified
CNN_2D_Model(
  (conv2d): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2d_hidden): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2d_hidden_layers): ModuleList(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (linear): Linear(in_features=3072, out_features=10, bias=True)
)
Number of parameters =  30982


CNN-CIFAR10/training-loss = 1.9543399810791016, model.n_iter=782, epoch=1: 100%|██████████| 782/782 [00:12<00:00, 62.06it/s]
CNN-CIFAR10/validation-loss = 1.7199288606643677, model.n_iter=157, epoch=1: 100%|████████| 157/157 [00:02<00:00, 72.44it/s]
CNN-CIFAR10/training-loss = 1.9425971508026123, model.n_iter=1564, epoch=2: 100%|█████████| 782/782 [00:12<00:00, 64.64it/s]
CNN-CIFAR10/validation-loss = 1.6346534490585327, model.n_iter=314, epoch=2: 100%|████████| 157/157 [00:01<00:00, 79.91it/s]
CNN-CIFAR10/training-loss = 2.3432414531707764, model.n_iter=2346, epoch=3: 100%|█████████| 782/782 [00:12<00:00, 64.67it/s]
CNN-CIFAR10/validation-loss = 1.656532883644104, model.n_iter=471, epoch=3: 100%|█████████| 157/157 [00:02<00:00, 71.54it/s]
CNN-CIFAR10/training-loss = 1.6092811822891235, model.n_iter=3128, epoch=4: 100%|█████████| 782/782 [00:12<00:00, 63.38it/s]
CNN-CIFAR10/validation-loss = 1.6673660278320312, model.n_iter=628, epoch=4: 100%|████████| 157/157 [00:02<00:00, 72.45it/s]
